# Understanding TensorRT - Convert Graph

## Convert the Model
1. checkpoint -> Frozen TensorFlow graph  
2. Frozen TensorFlow graph -> TFLite  
3a. TFLite -> EdgeTPU (Google Coral)  
3b. TFLite -> TensorRT (NVIDIA GPU)  

### Did you train locally or via SageMaker Docker?
- Locally - the training checkpoint is in code/model
- SageMaker - the training checkpoint was pushed to S3 then pulled back down and extracted to /trained_model

The conversion script expects it to be in /trained_model


## CUDA / TensorFlow Versions
### TF Lite
No big deal and there are probably no concerns here

### TensorRT 7.0
TensorRT 7.0 says the model must be trained w/ TensorFlow 1.14 (which requires CUDA 10.0).  
- did you train w/ 1.14  
- did you convert to tflite w/ 1.14  

just beware

### Edge TPU (Google Coral)
Hmm, this all worked with TF 1.13 and 1.14, haven't tested 1.15

In [ ]:
import os
import sys

In [ ]:
import tensorflow as tf

print (tf.__version__)

## GLOBALS

In [ ]:
# GLOBALS so work with the bash scripts
PROJECT_DIR = os.getcwd()
TASKS_DIR = os.path.join(PROJECT_DIR, 'tasks')

CODE_DIR = os.path.join(PROJECT_DIR, 'code')
TFLITE_DIR = os.path.join(PROJECT_DIR, 'tflite_model')
TENSORFLOW_DIR = os.path.join(PROJECT_DIR, 'tensorflow_model')
MODEL_RESEARCH_DIR = os.path.join(PROJECT_DIR, 'code/models/research')
SLIM_DIR = os.path.join(MODEL_RESEARCH_DIR, 'slim')
TRAINED_DIR = os.path.join(PROJECT_DIR, 'trained_model')

NUM_TRAINING_STEPS = 6000
# Local (non-SageMaker) -or- SageMaker
# PIPELINE_CONFIG = 'sagemaker_mobilenet_v1_ssd_retrain.config'
PIPELINE_CONFIG = 'local_mobilenet_v1_ssd_retrain.config'

# PICK 1 - A. Local or  B. Hosted

## A - Locally Trained
### get the checkpoint files to the correct location

In [ ]:
# is your checkpoint (NUM_TRAINING_STEPS) there?
! ls {CODE_DIR}/model/*{NUM_TRAINING_STEPS}*

In [ ]:
# copy to the /trained_model directory
# this will make it compatible with the convert script
! cp {CODE_DIR}/model/*{NUM_TRAINING_STEPS}* {TRAINED_DIR}
! ls {TRAINED_DIR}/*{NUM_TRAINING_STEPS}* -l

## check the configuration

In [ ]:
# PIPELINE_CONFIG = 'sagemaker_mobilenet_v1_ssd_retrain.config'
PIPELINE_CONFIG = 'local_mobilenet_v1_ssd_retrain.config'

# verify
! ls {CODE_DIR}/{PIPELINE_CONFIG}

## B - Trained in SageMaker

In [ ]:
! ls {TRAINED_DIR}/*{NUM_TRAINING_STEPS}* -l

In [ ]:
PIPELINE_CONFIG = 'sagemaker_mobilenet_v1_ssd_retrain.config'
# PIPELINE_CONFIG = 'local_mobilenet_v1_ssd_retrain.config'

# verify
! ls {CODE_DIR}/{PIPELINE_CONFIG}

# Convert the Checkpoint to Frozen Graph & TF Lite Graph

In [ ]:
# convert checkpoint is a task script - located in the tasks/ directory
os.chdir(TASKS_DIR)  
! ./convert_checkpoint_to_edgetpu_tflite.sh --checkpoint_num {NUM_TRAINING_STEPS} --pipeline_config {PIPELINE_CONFIG}


## Saved (output) Graphs

These models will now behave differently.   The API is different and the performance is different.   Furthermore the dependencies are different.  The JSON that is returned will not be compatible.   So, make sure you know what your doing.

### TensorFlow - Frozen Graph
Look for examples of using a Tensor frozen graph.  Look at notebook:  DetectModel_Step1_Local_TensorFlowFrozenGraph

### TensorFlow - TFLite
Look for examples of using TensorFlow Lite.   Look at notebook: DetectMOdel_Step1_Local_TFLite



In [ ]:
! ls {TENSORFLOW_DIR} -l

In [ ]:
! ls {TFLITE_DIR} -l